#  **PROJECT - SPRINT 7**

##### ***1.** Importamos el archivo games.csv y hacemos una visualizacion general de la taba para identificar valores ausentes y tipos de datos que la conforman*

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

#Vamos a leer el archivo
games_data = pd.read_csv('games.csv')

#Visualizar los tipos de datos que hay en la tabla
print(games_data.info())

##### ***2.** Calcular los porcentajes de valores nulos por cada columna para determinar como podemos tratarlos*

In [ ]:
# Porcentaje de valores nulos por columna
null_percentage = (games_data.isnull().sum() / len(games_data)) * 100
null_percentage

In [ ]:
# Pasar nombre de columnas a minusculas
games_data.columns = games_data.columns.str.lower()

# Eliminar filas con datos ausentes de las columnas name y year_of_release ya que esquivalian al 2% sobre el total de los datos
games_data.dropna(subset=['name'], inplace=True)
games_data.dropna(subset=['year_of_release'], inplace=True)

# Convertir la columna year_of_release en Int
games_data['year_of_release'] = games_data['year_of_release'].astype(int)

games_data['critic_score'] = games_data['critic_score'].fillna(games_data['critic_score'].mean())
#games_data['user_score'] = games_data['user_score'].astype(int)

#games_data['user_score'] = games_data['user_score'].replace({'tbd' , np.nan})
#games_data['user_score'] = games_data['user_score'].fillna(games_data['user_score'].mean())

games_data.info()